#  쿼리 확장 (Query Expansion)
--- 

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


`(4) 벡터스토어 로드`

In [4]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


`(5) 백터 검색기 생성`

In [5]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 5}
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 

---

## 쿼리 확장 (Query Expansion)

1. **Query Reformulation**
    - LLM을 사용하여 원래 질문을 다른 형태로 재작성하는 방식임
    - 동의어 확장, 질문 명확화, 관련 키워드 추가 등 다양한 방식으로 쿼리를 변형함
    - 검색의 다양성과 정확도를 향상시키는 특징이 있음

1. **Multi Query** 
    - Retriever에 지정된 LLM을 활용하여 원본 쿼리를 확장하는 방법임
    - 하나의 질문에 대해 다양한 관점과 표현으로 여러 개의 쿼리를 자동 생성함
    - LLM의 생성 능력을 활용하여 검색의 다양성과 포괄성을 향상시키는 특징이 있음

1. **Decomposition** 
    - 복잡한 질문을 여러 개의 단순한 하위 질문으로 분해하는 LEAST-TO-MOST PROMPTING 전략을 사용함
    - 각각의 하위 질문에 대해 개별적으로 검색을 수행하여 더 정확한 답변을 도출함
    - 복잡한 질문을 체계적으로 해결하면서 검색의 정확도를 높이는 특징이 있음

1. **Step-Back Prompting**
    - 주어진 구체적인 질문에서 한 걸음 물러나 더 일반적인 개념이나 배경을 먼저 검색함
    - 더 넓은 맥락에서 점차 구체적인 답변으로 좁혀가는 방식을 사용함
    - 복잡한 질문에 대해 더 포괄적이고 정확한 답변을 제공하는 특징이 있음

1. **HyDE (Hypothetical Document Embedding)**
    - 주어진 질문에 대해 가상의 이상적인 답변 문서를 LLM으로 생성함
    - 생성된 가상 문서를 임베딩하여 이를 기반으로 실제 문서를 검색하는 방식임
    - 질문의 맥락을 더 잘 반영한 검색이 가능한 특징이 있음

### 1) **Query Reformulation** 

- **Query Reformulation**은 **LLM**을 활용해 원본 질문을 다양한 형태로 재구성
- **동의어 확장**과 **키워드 추가**를 통해 검색 쿼리의 범위를 확장
- 모호한 질문을 **명확하게 구체화**하여 검색 정확도 향상
- 하나의 질문에 대해 **다양한 변형 쿼리**를 생성하여 검색 커버리지 확대

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_rewrite.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/abs/2305.14283

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 쿼리 리포뮬레이션을 위한 프롬프트 템플릿 정의
reformulation_template = """다음 질문을 검색 성능을 향상시키기 위해 다시 작성해주세요:
[질문]
{question}

다음 방식으로 질문을 재작성하세요:
1. 동의어 추가
2. 더 구체적인 키워드 포함
3. 관련된 개념 확장

[재작성된 질문]
"""

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(reformulation_template)

# LLM 모델 초기화
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

# 쿼리 리포뮬레이션 체인 생성
reformulation_chain = prompt | llm | StrOutputParser()

# 체인 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
reformulated_query = reformulation_chain.invoke({"question": query})

print(f"쿼리: {query}")
pprint(f"리포뮬레이션된 쿼리: \n{reformulated_query}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
('리포뮬레이션된 쿼리: \n'
 '[재작성된 질문]  \n'
 '리비안(Rivian)의 핵심 사업 경쟁력과 차별화된 강점은 무엇이며, 전기차 시장 내에서 경쟁 우위를 확보하는 주요 전략과 기술적 요소는 '
 '어떤 것들이 있나요? 또한, 리비안의 공급망 관리, 혁신 역량, 그리고 지속 가능성 측면에서의 경쟁력은 어떻게 평가되고 있나요?')


In [7]:
# 리포뮬레이션된 쿼리로 검색
retrieved_docs = chroma_k_retriever.invoke(reformulated_query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments였습니다.

**협력**

Rivian은 Alex Honnold, the Honnold Foundation, Casa Pueblo, Ewan McGregor, Charley Boorman, Yakima 및 MAXTRAX와 파트너십을 맺었습니다.

**소송**
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 됨.
- 2021년 11월, Rivian은 IPO를 통해 135억 달러를 조달하여 상장 회사가 됨.
- Ford와 Rivian은 전기 자동차 공동 개발 계획을 취소.
- 2022년 3월, Rivian은 부

In [8]:
# Runnable 객체로 변환하여 검색기 생성 (LCEL)
reformulation_retriever = reformulation_chain | chroma_k_retriever

# 쿼리 리포뮬레이션 검색기 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = reformulation_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments였습니다.

**협력**

Rivian은 Alex Honnold, the Honnold Foundation, Casa Pueblo, Ewan McGregor, Charley Boorman, Yakima 및 MAXTRAX와 파트너십을 맺었습니다.

**소송**
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체가 됨.
- 2021년 11월, Rivian은 IPO를 통해 135억 달러를 조달하여 상장 회사가 됨.
- Ford와 Rivian은 전기 자동차 공동 개발 계획을 취소.
- 2022년 3월, Rivian은 부

### 2) **Multi Query** 

- **Multi Query**는 **Retriever의 LLM**을 사용해 단일 질문을 다수의 쿼리로 확장
- 원본 질문에 대해 **다양한 관점**과 **표현 방식**으로 쿼리 자동 생성
- **LLM의 생성 능력**을 활용해 검색 범위를 자연스럽게 확장
- 검색의 **다양성**과 **포괄성**이 향상되어 관련 문서 검색 확률 증가


<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/multi-query.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>

[출처] https://arxiv.org/abs/2411.13154

`(1) MultiQueryRetriever 활용`

- https://python.langchain.com/docs/how_to/MultiQueryRetriever/

In [9]:
# 멀티 쿼리 생성
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 모델 초기화 (멀티 쿼리 생성용)
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.7,
)

# 기본 retriever를 이용한 멀티 쿼리 생성 
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=chroma_k_retriever, llm=llm
)

query = "리비안의 사업 경쟁력은 어디서 나오나요?"
retrieved_docs = multi_query_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Inve

`(2) Custom Prompt 활용`

In [10]:
from typing import List

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 출력 파서: LLM 결과를 질문 리스트로 변환
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        """Split the text into lines and remove empty lines."""
        return [line.strip() for line in text.strip().split("\n") if line.strip()]
    

# 쿼리 생성 프롬프트
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Generate three different versions of the given user question to retrieve relevant documents from a vector database. The goal is to reframe the question from various perspectives to overcome limitations of distance-based similarity search.

    The generated questions should have the following characteristics:
    1. Maintain the core intent of the original question but use different expressions or viewpoints.
    2. Include synonyms or related concepts where possible.
    3. Slightly broaden or narrow the scope of the question to potentially include diverse relevant information.

    Write each question on a new line and include only the questions.

    [Original question]
    {question}
    
    [Alternative questions]
    """,
)

# 멀티쿼리 체인 구성
multiquery_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
result = multiquery_chain.invoke({"question": query})

print("생성된 대안 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

생성된 대안 질문들:
1. 리비안이 시장에서 경쟁 우위를 가지는 주요 요소는 무엇인가요?
2. 리비안의 핵심 강점과 비즈니스 성공 요인은 어떤 부분에서 비롯되나요?
3. 리비안이 다른 기업과 차별화되는 경쟁력의 원천을 설명해 주세요.


In [11]:
# 다중 쿼리 검색기 생성
multi_query_custom_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever, # 기본 retriever
    llm_chain=multiquery_chain,   # 멀티쿼리 체인
    parser_key="lines"            # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_custom_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Inve

# ~ 14:50분까지 실습

### 3) **Decomposition** 

- **단계별 분해 전략**을 통해 복잡한 질문을 작은 단위로 나누어 처리함
- 각 하위 질문마다 **독립적인 검색 프로세스**를 진행하여 정확도를 향상시킴
- **LEAST-TO-MOST PROMPTING**을 활용하여 체계적인 문제 해결 방식을 구현함
- 복잡한 문제를 단순화하여 검색 효율성을 극대화하는 방법론

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_decomposition.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/pdf/2205.10625

In [12]:
from langchain.prompts import PromptTemplate
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to decompose the given input question into multiple sub-questions. 
    The goal is to break down the input into a set of sub-problems/sub-questions that can be answered independently.

    Follow these guidelines to generate the sub-questions:
    1. Cover various aspects related to the core topic of the original question.
    2. Each sub-question should be specific, clear, and answerable independently.
    3. Ensure that the sub-questions collectively address all important aspects of the original question.
    4. Consider temporal aspects (past, present, future) where applicable.
    5. Formulate the questions in a direct and concise manner.

    [Input question] 
    {question}

    [Sub-questions (5)]
    """,
)

# 쿼리 생성 체인
decomposition_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
result = decomposition_chain.invoke({"question": query})

print("생성된 서브 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

생성된 서브 질문들:
1. 1. 리비안의 주요 사업 분야는 무엇인가요?
2. 2. 리비안이 경쟁사와 차별화되는 기술적 강점은 무엇인가요?
3. 3. 리비안의 생산 및 공급망 관리 전략은 어떻게 구성되어 있나요?
4. 4. 시장에서 리비안의 현재 입지와 성과는 어떤가요?
5. 5. 리비안이 앞으로 강화하려는 경쟁력 요소는 무엇인가요?


In [13]:
# 다중 쿼리 검색기 생성
multi_query_decompostion_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever,    # 기본 retriever
    llm_chain=decomposition_chain,   # 서브 질문 생성 체인
    parser_key="lines"               # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_decompostion_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사

### 4) **Step-Back Prompting**

- **단계적 후퇴 방식**을 통해 구체적 질문을 일반적 맥락에서 접근함
- **맥락 기반 검색**으로 넓은 관점에서 구체적 답변으로 좁혀나감
- **포괄적 접근법**을 활용하여 복잡한 질문에 대한 이해도를 높임
- 일반적 맥락에서 시작하여 구체적 해답을 찾아가는 체계적 접근 방식

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_stepback.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/pdf/2310.06117

`(1) Step-Back 질문 생성`

In [14]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Few Shot 예제 - (구체적 질문, 포괄적 질문) 쌍
examples = [
    {
        "input": "애플의 M1 칩 개발이 기업 가치에 미친 영향은?",
        "output": "기업의 핵심 기술 내재화가 경쟁우위에 미치는 영향은 무엇인가?",
    },
    {
        "input": "아마존의 AWS가 수익성에 기여하는 방식은?",
        "output": "기업의 새로운 사업 영역 확장이 수익 구조에 미치는 영향은 무엇인가?",
    },
    {
        "input": "토요타의 하이브리드 기술 전략의 핵심은?",
        "output": "자동차 산업에서 친환경 기술 혁신이 기업 성장에 미치는 영향은 무엇인가?",
    }
]

# 프롬프트 템플릿 초기화
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Step-Back 생성을 위한 프롬프트
step_back_prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """당신은 기업 분석 전문가입니다. 특정 기업에 대한 구체적인 질문을 해당 산업이나 비즈니스 전반의 일반적인 관점에서 
                재해석하는 것이 임무입니다. 산업 동향, 경쟁 구도, 기술 혁신, 사업 모델 등의 관점에서 더 포괄적인 질문으로 
                바꾸어 주세요. 다음은 예시입니다:"""
            ),
            few_shot_prompt,
            ("user", "{question}"),
        ])

# Step-Back 체인 생성
step_back_chain = step_back_prompt | llm | StrOutputParser()

# Step-Back 질문 생성
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
step_back_question = step_back_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"Step-Back 질문: {step_back_question}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
Step-Back 질문: 신생 전기차 기업이 기존 완성차 업체와 차별화된 경쟁우위를 확보하는 전략은 무엇인가?


In [15]:
# Step-Back 검색기 생성
step_back_retriever = step_back_chain | chroma_k_retriever

# Step-Back 검색 실행
retrieved_docs = step_back_retriever.invoke({"question": query})

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하는 것입니다. Tesla는 자동차의 하드웨어를 지속적으로 업데이트하고 웹사이트와 회사 소유 매장을 통해 직접 차량을 판매합니다. Tesla는 수직적으로 통합되어 많은 구성 요소를 자체 개발합니다. Tesla는 일반적으로 지속 가능한 에너지 채택을 촉진하기 위해 경쟁 업체가 자사 기술을 라이선스하도록 허용합니다.

## 기술

### 배터리

Tesla는 CATL, LG Energy Solution 및 Panasonic에서 공급받은 원통형 및 각형 배터리 셀을 사용하고 있으며 자체 배터리를 생산할 수 있는 능력을 구축하고 있습니다.
</Document>
<Source>이 문서는 미국 전기차 회사인 '테슬라'에 대한 문서입니다.</Source> [출처: data\테슬라_KR.md]
<Document>
**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.

| 연도 | 수익 (백만 USD) | 순이익 (백만 USD) | 총 자산 (백만 USD) |
| ---- | --------------- | ----------------- | ------------------ |
| 2020 | 0               | -1,018            | 4,602              |
| 2021 | 55              | -4,688            | 22,294             |
| 2022 | 1,658           | -6,752            | 17,876             |
| 2023 | 4,434           | -5,432            | 16,778             |

**최대 주주**

2023년 12월 현재 최대 주주는 Amazon, T. Rowe Price International, The Va

`(2) 최종 답변 생성`

In [16]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate


# 프롬프트 템플릿 초기화
response_prompt = ChatPromptTemplate.from_template(
            """당신은 전문가입니다. 다음 컨텍스트와 질문을 바탕으로 포괄적인 답변을 제공해주세요.

            일반 컨텍스트:
            {normal_context}
            
            기본 개념 컨텍스트:
            {step_back_context}
            
            원래 질문: {question}
            
            답변:"""
        )

# 문서 포맷팅 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# 답변 생성 체인
answer_chain = (
            {
                "normal_context": chroma_k_retriever,
                "step_back_context": step_back_retriever,
                "question": RunnablePassthrough(),
            }
            | response_prompt
            | llm
            | StrOutputParser()
        )

# 답변 생성
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
answer = answer_chain.invoke(query)

print(f"쿼리: {query}")
print(f"답변: {answer}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
답변: 리비안(Rivian)의 사업 경쟁력은 여러 가지 측면에서 도출됩니다. 전기차 산업 내에서 독자적인 강점과 전략적 차별화를 통해 시장에서 경쟁 우위를 확보하려는 노력이 반영되어 있습니다. 이를 구체적으로 분석하면 다음과 같습니다.

1. **전기차 및 배터리 전문성**  
   리비안은 2009년 설립 이후 전기 자동차와 배터리 기술 개발에 집중해오면서, 전기 픽업트럭(R1T)과 전기 SUV(R1S) 등 독자적인 제품라인을 구축했습니다.  
   - 2021년에는 완전 전기 픽업트럭 R1T를 처음으로 소비자에게 배송하며 해당 시장에서 선도적인 위치를 확보했습니다.  
   - 배터리 및 전기차 충전 서비스, 자동차 보험 등 전기차 생태계 전반에 걸친 서비스를 함께 제공하는 점도 차별화 요소입니다.

2. **특화된 제품 전략 – 아웃도어 및 레크리에이션 분야 집중**  
   리비안은 단순한 전기차 제조를 넘어 야외 레크리에이션, 모험 활동에 적합한 라이프스타일 브랜드로 포지셔닝하고 있습니다. 이는 테슬라 등 경쟁사와 차별화되는 중요한 경쟁력입니다.  
   - 아웃도어 활동에 최적화된 차량 설계와 내구성, 기능성을 강조함으로써 틈새시장 공략에 성공하고 있습니다.  
   - Alex Honnold(클라이머), Casa Pueblo(환경단체) 등과의 파트너십은 브랜드 이미지 강화와 시장 내 신뢰 확보에 기여합니다.

3. **통합된 생산 및 엔지니어링 시설 운영**  
   리비안은 미국 내 주요 거점(캘리포니아 어바인, 일리노이 노멀, 미시간 플리머스 등)에 엔지니어링, 제조, 소프트웨어 개발 시설을 분산 운영하며, 자체 생산 및 품질 관리 역량을 강화하고 있습니다.  
   - 특히 차량 부품 생산 및 조립을 직접 수행하여 공급망 리스크 관리 및 제품 품질 향상에 집중하고 있습니다.  
   - 배터리 및 조립 공장 신설(50억 달러 규모 예정)은 향후 생산능력 확장 및 원가 경쟁력 제고의 기반이 될 전

### 5) **HyDE** (Hypothetical Document Embedding)

- **가상 문서 생성**을 통해 주어진 질문에 대해 가상의 이상적인 답변 문서를 LLM으로 생성함
- 생성된 문서의 **임베딩 기반 검색**으로 실제 문서와 매칭을 수행함
- **맥락 기반 검색 방식**으로 질문의 의도를 더 정확하게 반영함

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/query_HyDE.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


[출처] https://arxiv.org/abs/2212.10496

`(1) 가상 문서 생성`

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# HyDE를 위한 프롬프트 템플릿 생성
template = """주어진 질문에 대한 이상적인 문서 내용을 생성해주세요.
문서는 학술적이고 전문적인 톤으로 작성되어야 합니다.

질문: {question}

문서 내용:"""

hyde_prompt = ChatPromptTemplate.from_template(template)

# LLM 모델 초기화
hyde_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 문서 생성 체인 생성
hyde_chain = hyde_prompt | hyde_llm | StrOutputParser()

# 문서 생성 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
hypothetical_doc = hyde_chain.invoke({"question": query})

print(f"쿼리: {query}")
print(f"문서 내용: {hypothetical_doc}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
문서 내용: 리비안(Rivian)의 사업 경쟁력은 다각적인 요소에서 기인하며, 전기차(EV) 시장 내에서 독자적인 위치를 확보하는 데 중요한 역할을 하고 있다. 본 문서에서는 리비안의 경쟁력을 구성하는 주요 요인들을 기술적 혁신, 제품 포트폴리오, 공급망 관리, 브랜드 전략 및 지속 가능성 측면에서 분석하고자 한다.

첫째, 리비안의 기술적 혁신은 동종 업계 내에서 두드러진 경쟁력의 원천이다. 리비안은 자체 개발한 전기차 플랫폼 ‘스케이트보드(Skateboard)’ 아키텍처를 기반으로, 배터리 팩, 전기 모터, 전력 전자장치 등을 통합하여 높은 주행 성능과 효율성을 구현하고 있다. 특히, 4개의 독립 전기 모터를 통한 정밀한 토크 벡터링(torque vectoring) 기술은 오프로드 및 다양한 주행 환경에서 우수한 주행 안정성과 민첩성을 제공한다. 또한, 소프트웨어 중심의 차량 제어 시스템과 OTA(Over-The-Air) 업데이트 기능을 통해 지속적인 성능 개선과 사용자 경험 향상이 가능하다.

둘째, 리비안은 전기 픽업트럭과 SUV 시장에 집중한 제품 포트폴리오를 통해 틈새시장을 공략하고 있다. 전통적으로 내연기관 차량 중심이었던 픽업트럭 시장에 전기차를 성공적으로 도입함으로써, 경쟁사 대비 차별화된 시장 진입 전략을 구사하고 있다. 특히, 미국 내 아웃도어 및 레저 활동에 특화된 소비자층을 겨냥한 디자인과 기능성은 브랜드 충성도를 높이는 데 기여한다.

셋째, 리비안은 전략적 공급망 관리와 생산 역량 강화에 주력하고 있다. 배터리 셀 및 핵심 부품의 안정적 확보를 위해 주요 배터리 제조사와의 협력 관계를 구축하고 있으며, 자체 생산 시설 확장과 자동화 공정 도입을 통해 생산 효율성을 제고하고 있다. 이러한 공급망 통제력은 원가 경쟁력 확보와 제품 출시 일정 준수에 중요한 역할을 한다.

넷째, 브랜드 전략 측면에서 리비안은 친환경성과 혁신 이미지를 강조하며, 지속 가능성에 대한 소비자 인식을 적극적으로 반

`(2) 유사 문서 검색`

In [18]:
# 가상 문서를 기반으로 실제 문서 검색
    
retrieved_docs = chroma_k_retriever.invoke(hypothetical_doc)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

<Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
**시설**

- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 둔 본사.
- **Normal, Illinois:** 차량 부품을 생산하고 조립을 수행하는 제조 공장.
- **Plymouth, Michigan:** 차량 엔지니어링, 프로토타입 제작, 공급망 및 회계에 중점을 둡니다.
- **Palo Alto, California:** 소프트웨어 개발 및 엔지니어링에 중점을 둡니다.
- Carson, California 및 Woking, England에 추가 사무실이 있습니다.
- 애틀랜타 동쪽에 있는 새로운 50억 달러 규모의 배터리 및 조립 공장은 보류 중입니다.

**재정**

Rivian의 재무 성과는 상당한 수익 성장과 상당한 순손실로 특징지어집니다.
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
<Document>
## 비즈니스 전략

Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하

`(3) 최종 답변 생성`

In [19]:
# 최종 RAG를 위한 프롬프트 템플릿 생성
template = """다음 컨텍스트를 바탕으로 질문에 답변해주세요:

컨텍스트:
{context}

질문: {question}

답변:"""

rag_prompt =  ChatPromptTemplate.from_template(template)

# RAG 체인 생성
rag_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
rag_chain = rag_prompt | rag_llm | StrOutputParser()
    
# RAG 실행
query = "리비안의 사업 경쟁력은 어디서 나오나요?"
context = format_docs(retrieved_docs)

answer = rag_chain.invoke({"context": context, "question": query})

print(f"쿼리: {query}")
print(f"답변: {answer}")

쿼리: 리비안의 사업 경쟁력은 어디서 나오나요?
답변: 리비안의 사업 경쟁력은 다음과 같은 요소들에서 비롯됩니다:

1. **전기 픽업 트럭 및 SUV 시장의 선도적 위치**  
   리비안은 2021년 9월 R1T 전기 픽업 트럭 배송을 시작하며 완전 전기 픽업을 소비자 시장에 처음으로 출시한 제조업체가 되었습니다. 이는 전기차 시장 내에서 틈새이자 성장 가능성이 큰 세그먼트에서의 선점 효과를 가져옵니다.

2. **다양한 제품 및 서비스 포트폴리오**  
   전기 자동차뿐만 아니라 배터리, 전기차 충전 서비스, 자동차 보험 등 관련 생태계를 구축하여 고객에게 통합된 경험을 제공합니다. 이는 고객 충성도 강화와 추가 수익 창출에 기여합니다.

3. **전략적 생산 및 엔지니어링 인프라**  
   미국 내 여러 지역에 걸친 전문화된 시설(캘리포니아 어바인 본사, 일리노이 노멀 제조 공장, 미시간 플리머스 엔지니어링 및 공급망, 캘리포니아 팔로알토 소프트웨어 개발 등)을 통해 제품 개발과 생산의 효율성을 높이고 있습니다.

4. **혁신적인 기술 개발 역량**  
   자체 배터리 및 전기차 기술 개발에 집중하며, 특히 배터리 및 차량 설계에 강점을 두고 있습니다. 이는 제품 경쟁력과 원가 절감에 중요한 역할을 합니다.

5. **상장 및 자본 조달 능력**  
   2021년 IPO를 통해 135억 달러를 조달하며 대규모 투자와 확장에 필요한 자본을 확보했습니다. 이를 바탕으로 생산 능력 확대 및 신기술 개발에 투자할 수 있습니다.

6. **차별화된 브랜드 이미지 및 시장 포지셔닝**  
   전기 픽업 트럭과 SUV라는 특화된 제품군으로 친환경 모험형 차량 시장을 공략하며, Tesla 등 기존 전기차 업체와 차별화된 고객층을 확보하고 있습니다.

이처럼 리비안은 제품 혁신, 생산 인프라, 서비스 확장, 자본력, 그리고 시장 선점 전략을 통해 전기차 시장에서 경쟁력을 확보하고 있습니다.


`(4) HyDE 체인 종합`

In [20]:
# Step 1. 가상 문서 생성
query = "테슬라의 경영진을 분석해주세요."

hypothetical_doc = hyde_chain.invoke({"question": query})

# Step 2. 유사 문서 검색
retrieved_docs = chroma_k_retriever.invoke(hypothetical_doc)

# Step 3. 최종 답변 생성
final_answer = rag_chain.invoke(
    {
        "context": format_docs(retrieved_docs), 
        "question": query
    }
)

print(f"쿼리: {query}")
print(f"가상 문서 내용: {hypothetical_doc}")
print(f"답변: {final_answer}")

쿼리: 테슬라의 경영진을 분석해주세요.
가상 문서 내용: ### 테슬라 경영진 분석

#### 1. 서론
테슬라(Tesla, Inc.)는 전기자동차 및 에너지 저장 솔루션 분야에서 혁신을 주도하는 글로벌 기업으로, 그 성공의 배경에는 뛰어난 경영진의 리더십과 전략적 의사결정이 자리잡고 있다. 본 문서에서는 테슬라의 주요 경영진 구성과 이들의 경영 스타일, 전략적 기여도를 분석함으로써 기업 성과에 미친 영향을 고찰하고자 한다.

#### 2. 경영진 구성 및 주요 인물

##### 2.1 일론 머스크 (Elon Musk) – 최고경영자(CEO) 및 제품 아키텍트
일론 머스크는 테슬라의 창립 초기부터 핵심적인 역할을 수행해온 인물로, 현재 CEO 및 제품 아키텍트로서 회사의 비전과 전략을 총괄한다. 그의 리더십은 혁신 중심의 기업 문화를 조성하고, 전기차 시장에서의 기술적 우위를 확보하는 데 결정적이었다. 머스크는 공격적인 목표 설정과 위험 감수를 통해 신제품 개발과 생산 확대를 가속화하며, 지속 가능한 에너지 전환을 위한 장기적 비전을 제시한다.

##### 2.2 자비에르 드 실바 (Xavier De Silva) – 최고운영책임자(COO)
자비에르 드 실바는 테슬라의 글로벌 생산 및 운영을 책임지는 COO로서, 공급망 관리와 제조 공정의 효율성 제고에 중점을 둔다. 그의 경험은 대규모 생산 능력 확장과 품질 관리에 기여하며, 테슬라가 급격한 수요 증가에 대응할 수 있도록 지원한다.

##### 2.3 잭슨 데이비스 (Zachary Kirkhorn) – 최고재무책임자(CFO)
잭슨 데이비스는 재무 전략과 자본 조달을 담당하며, 테슬라의 재무 건전성 유지와 투자자 신뢰 확보에 중요한 역할을 한다. 그의 재무 관리 하에 테슬라는 공격적인 연구개발 투자와 글로벌 시장 확장을 위한 자금 조달을 성공적으로 수행하고 있다.

##### 2.4 드류 배글리 (Drew Baglino) – 최고기술책임자(CTO)
드류 배글리는 테슬라의 기술 혁신을 이끄는 CTO로, 배터리 기술